In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

In [ ]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

In [191]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-03-26 16:40:02,041 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:85:6a:10:3c
2024-03-26 16:40:04,784 | INFO     |: Connected to c0:83:85:6a:10:3c


In [ ]:
# Disconnect
ble.disconnect()

In [ ]:
ble.send_command(CMD.MOVE_FORWARD_TIMED, "30|2000")

In [192]:
ble.send_command(CMD.MOVE_WHEELS_TIMED, "50|0|2000")

# L_min : 
# R_min : 102

2024-03-26 16:40:15,880 | INFO     |: Disconnected from D521BD93-F974-CFE7-133C-181108BA8274


ble.send_command(CMD.ADJUST_PWM_VALS, "0|255|0|255|0.2|0.0|-0.2")

# 255|90 works at move forward speed 40

In [ ]:
stamped_data.clear()
ble.send_command(CMD.PID_YAW, "50|50|255|160|0.2|0.0|0.0")

In [168]:
def notification_handler(uuid, char_val_array):
    str_data = char_val_array.decode()[2:]
    print(str_data)

ble.start_notify(ble.uuid['RX_STRING'], notification_handler)

In [ ]:
stamped_data = []

def notification_handler(uuid, char_val_array):
    str_data = char_val_array.decode()[2:]
    data = str_data.split(',')
    stamped_data.append(data)

ble.start_notify(ble.uuid['RX_STRING'], notification_handler)

In [ ]:
time_data = []
gyr_data = []
yaw_data = []
pd_data = []
for data_list in stamped_data:
    assert(len(data_list) >= 1)
    time_data.append((int(data_list[0]) - int(stamped_data[0][0])) / 1000.)
    gyr_data.append(float(data_list[1]))
    yaw_data.append(float(data_list[2]))
    pd_data.append(float(data_list[3]))

print("raw data len: " + str(len(time_data)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft

time = np.array(time_data)
gyr = np.array(gyr_data)
yaw = np.array(yaw_data)
pd = np.array(pd_data)

plt.plot(time, gyr, 'r')
plt.plot(time, yaw, 'b')
plt.plot(time, pd, 'g')
plt.show()